# JSON_VALUE and JSON_QUERY Functions

## Extracting Individual JSON Elements

Db2 provides two scalar functions that allow you to extract individual JSON elements from a JSON document in either their native JSON appearance or as an SQL value ready for use in a relational application or SQL statement.
* `JSON_VALUE` function is used to retrieve a single SQL value from a JSON document
* `JSON_QUERY` function is used to retrieve a single JSON value or JSON object from a JSON document

## JSON_VALUE: Retrieving Individual Values 
The `JSON_VALUE` function is used to retrieve a single value from a JSON document in the form of a "native" SQL data type which can be directly referenced by a user application like any other SQL data value or it can be embedded within another SQL statement. 

This function implicitly converts the returning value from its original JSON format to the identified Db2 data type. Since it is a scalar function, `JSON_VALUE` will only return a single value and, if the evaluation of the JSON path expression results multiple JSON values being returned, `JSON_VALUE` will return an error.

***JSON_VALUE Syntax***
![JSON_VALUE](images/JSON_VALUE.png)

***Returning Clause***
![JSON_VALUE](images/jv-returning-clause.png)

***Empty and Error Clause***
![JSON_VALUE](images/jv-empty-clause.png)

### JSON Expression and JSON Path expression
The *json-expression* and *json-path-expression* were discussed in previous notebooks.

### RETURNING Clause
The `RETURNING` clause is an optional part of the `JSON_VALUE` function and indicates what SQL data type should be used to format the JSON value retrieved by the function. If you want to have the results returned as a specific data type, then you need to supply this parameter otherwise  Db2 will return a large character field (CLOB). 

The `RETURNING` clause can contain any of the data types that are supported within Db2. You must ensure that the size of the output data type is large enough to support the data being retrieved, and that it is of the proper type. 
The following book document is used to illustrate the use of the `JSON_VALUE` function.

In [ ]:
book = {
   "authors": [{"first_name": "Paul",  "last_name" : "Bird"},
               {"first_name": "George","last_name" : "Baklarz"}],
   "foreword": {
              "primary": {
                          "first_name": "Thomas",
                          "last_name" : "Hronis"
                         }
              },
   "formats": ["Hardcover","Paperback","eBook","PDF"]
}

### Load Db2 Extensions and Connect to the Database
The `connection` notebook contains the `CONNECT` statement which allows access to the `SAMPLE` database. If you need to modify the connection information, edit the `connection.ipynb` notebook.

In [ ]:
%run ../db2.ipynb
%run ../connection.ipynb

To retrieve the first book format, we must include the array element number.

In [ ]:
%sql VALUES JSON_VALUE(:book,'$.formats[0]');

You can specify how you want to return this value by providing a `RETURNING` clause. This example requests that the string be returned as VARCHAR(100):

In [ ]:
%sql VALUES JSON_VALUE(:book,'$.formats[0]' RETURNING VARCHAR(100))

A good practice is to specify the `RETURNING` clause for values that you know the exact type for, and you should always use the `RETURNING` clause to convert numeric strings into proper SQL data types. For example, consider a phone extension that is stored as a string in the JSON document:

In [ ]:
%sql VALUES JSON_VALUE('{"extension": "000010"}','$.extension');

The `JSON_VALUE` function will return this value as a character string by default (since it is returning it as value compatible with a CLOB). If you want this returned as an integer, then you will need to add the `RETURNING INT` clause to the function.

In [ ]:
%sql VALUES JSON_VALUE('{"extension": "000010"}','$.extension' RETURNING INT)

### ON EMPTY and ON ERROR Clause
When an empty or error condition is encountered, Db2 will raise one of two exceptions: `ON EMPTY` or `ON ERROR`. Which condition fires is dependent on the use of the `lax` and `strict` keywords in the path expression (details can be found in the notebook that discussed `JSON_EXISTS`)

The actions for these two exception handling clauses are:
* NULL – Return a null value instead of an error
* ERROR – Raise an error
* DEFAULT <value> – Return a default value instead
    
These actions are specified in front of the error handling clause. The default value is `NULL ON EMPTY` and `NULL ON ERROR`. The other option for handling missing values is to return a default value using the `DEFAULT` clause. 

This option allows the function to return a value rather than a null. 

![JSON_VALUE](images/DEFAULT.png)


The following SQL will return a default value when a middle name cannot be found in the document.

In [ ]:
%%sql
VALUES JSON_VALUE(:book,'lax $.foreword.primary.middle_name' 
   DEFAULT 'No middle initial' ON EMPTY);

When using the `DEFAULT` clause, make sure to include a `RETURNING` clause which matches the data type of default value to avoid conversion errors.

### Examples
The following book document will be used to illustrate how the `JSON_VALUE` function is used.

In [ ]:
book = {
  "authors": 
    [
      {"first_name": "Paul",  "last_name" : "Bird"},
      {"first_name": "George","last_name" : "Baklarz"}
    ],
  "foreword": 
    {
      "primary": {"first_name": "Thomas","last_name" : "Hronis"}
    },
  "formats":
    {
      "hardcover": 19.99,
      "paperback":  9.99,
      "ebook"    :  1.99,
      "pdf"      :  1.99
    }
}

***Retrieve the First Name of the Author of the Foreword Section***

The dot notation is used to traverse along the foreword object and retrieve the first name of the primary author. 

In [ ]:
%sql VALUES JSON_VALUE(:book,'$.foreword.primary.first_name')

***Retrieve the Last Name of the Second Author of the Book***

Indexes start at zero so the second author would require a value of 1 in the index field.

In [ ]:
%sql VALUES JSON_VALUE(:book,'$.authors[1].last_name')

***What is the Cost of a Paperback Version of the Book?***

Using the `JSON_VALUE` function with the `RETURNING` clause will return all values as character strings (CLOB).

In [ ]:
%sql VALUES JSON_VALUE(:book,'$.formats.paperback');

If you want to retrieve the data as a numeric value, then you must use the `RETURNING` clause.

In [ ]:
%sql VALUES JSON_VALUE(:book,'$.formats.paperback' RETURNING DEC(9,2));

***Get the Last Name of the Foreword Author using Array Notation***

The following statement will work with the default lax mode since Db2 will ignore the structural problems (specifying an index when there is 
no array).

In [ ]:
%sql VALUES JSON_VALUE(:book,'$.foreword[0].primary.last_name');

Switching to strict mode will cause the statement to fail and return a `NULL` value (`NULL ON ERROR` is the default).

In [ ]:
%sql VALUES JSON_VALUE(:book,'strict $.foreword[0].primary.last_name')

## JSON_QUERY: Retrieving Objects and Arrays
Because `JSON_VALUE` is a scalar function which is returning values using native Db2 data types, it is limited to retrieving atomic or individual values from within a document. In order to extract native JSON values, which can include complex ones such as multiple array values or entire JSON objects, you must use the `JSON_QUERY` function. The `JSON_QUERY` function has a similar syntax as JSON_VALUE but adds some modifiers to handle complex results such as arrays. 

This function implicitly returns values in their original JSON or BSON format. Since it is a scalar function, `JSON_QUERY` can only return a single JSON value as its result; if the evaluation of the JSON path expression results multiple, independent JSON values being returned, `JSON_QUERY` will process this as an error.

***JSON_QUERY Syntax***
![JSON_QUERY](images/JSON_QUERY.png)

***Returning Clause***
![JSON_QUERY](images/jq-returning-clause.png)

***Wrapper Clause***
![JSON_QUERY](images/jq-wrapper-clause.png)

***Quotes Clause***
![JSON_QUERY](images/jq-quotes-clause.png)

***Empty and Error Clause***
![JSON_QUERY](images/jq-empty-clause.png)

### JSON Expression and JSON Path Expression
The *json-expression* and *json-path-expression* were discussed in previous notebooks.

Note that the `ARRAY` keyword in the wrapper clause is not required when using the `JSON_QUERY` function but is included to maintain ISO SQL compatibility. 

### RETURNING Clause
The `RETURNING` clause is an optional part of the `JSON_QUERY` function. By default, the `JSON_QUERY` function returns the value as a character string (or binary if you choose `FORMAT BSON`). If you are extracting an individual value, then you can only specify a character or binary data type with the `RETURNING` clause. `JSON_VALUE` provides more flexibility with individual values, so if you need to return a value as something other than a character string, you should consider using that function instead.

The `RETURNING` clause includes the optional `FORMAT JSON` or `FORMAT BSON` specification. This clause will tell the function to return the values as a character JSON value or convert it to a binary BSON format. If the `FORMAT BSON` clause is used, the data type in the `RETURNING` clause must be binary (`VARBINARY`, `BLOB`). 

The following query will retrieve the price of the paperbook version of a book.

In [ ]:
%sql VALUES JSON_QUERY(:book,'$.formats.paperback');

While the result above looks like a number, it is actually a character string, and trying to do any calculations on it will produce an error.

In [ ]:
%sql VALUES JSON_QUERY(:book,'$.formats.paperback') * 10;

### Wrappers
`JSON_QUERY` can be used to retrieve individual values from a document, but its strength is in handling more complex types. `JSON_QUERY` function has the ability to return multiple JSON values as a single JSON object through the use of the array wrapper clause. This clause allows you to "wrap" multiple values returned from the JSON document into a single JSON array type. There are three options when dealing with wrapping results:
* WITHOUT (ARRAY) WRAPPER
* WITH CONDITIONAL (ARRAY) WRAPPER 
* WITH UNCONDITIONAL (ARRAY) WRAPPER

The `WITHOUT` clause is the default setting which means that the results will not be wrapped as an array regardless of how many JSON values are returned. If the result of your search is more than one value, the function will treat this as an error and follow the behavior set in the `ON ERROR` clause (which is `NULL` by default).

The two other options will create an `ARRAY WRAPPER` based on the number of values returned. An `UNCONDITIONAL WRAPPER` will always create an array of values, while a `CONDITIONAL WRAPPER` will only create an array if there are one or more elements returned or if it is an object. If the result is an array, it will not place an array wrapper around it. 

To demonstrate the way the `WRAPPER` clause is handled, the following formats and primary document snippets will used.

In [ ]:
formats = {"formats": ["Hardcover","Paperback","eBook","PDF"]}
primary = {"primary": {"first_name": "Thomas","last_name" : "Hronis"}}

***Retrieve the Formats Array (Default Settings)***

The entire contents of an object, array, or an individual value can be retrieved with the `JSON_QUERY` function without the need of a wrapper. The following statement will retrieve the complete contents of the formats array. This is possible a JSON array is itself considered a single JSON value, the value associated with the "formats" key, even though that value is a complex one.

In [ ]:
%sql -j VALUES JSON_QUERY(:formats, '$.formats' WITHOUT WRAPPER);

Adding the `WITH CONDITIONAL WRAPPER` clause will ensure that the results will be wrapped as an array `[]` if it is required. The following SQL will return the same results as the previous example that did not specify a wrapper. Since the object is already a JSON array, there is no need to place the array characters around it.

In [ ]:
%sql -j VALUES JSON_QUERY(:formats, '$.formats' WITH CONDITIONAL WRAPPER);

The final example uses the `UNCONDITIONAL ARRAY WRAPPER` which will force the `JSON_QUERY` function to add the array wrapper around the result, regardless of the type of data being returned.

In [ ]:
%sql -j VALUES JSON_QUERY(:formats, '$.formats' WITH UNCONDITIONAL WRAPPER);

***Retrieve ALL Formats in the Document***

Using the asterisk (`*`) in a path expression will normally result in multiple keywords matching and thus, multiple independent JSON values being returned. If you do not specify a wrapper setting, `JSON_QUERY` will default to `WITHOUT WRAPPER` and an error is assumed. In this example, the `NULL` value will be returned as the default setting is `NULL ON ERROR`.

In [ ]:
%sql VALUES JSON_QUERY(:formats, '$.formats[*]' WITHOUT WRAPPER);

Adding `WITH CONDITIONAL` or `WITH UNCONDITIONAL` to the function will result in an array containing all the independent values as elements being returned by the function.

In [ ]:
%sql -j VALUES JSON_QUERY(:formats, '$.formats[*]' WITH CONDITIONAL WRAPPER)

In [ ]:
%sql -j VALUES JSON_QUERY(:formats, '$.formats[*]' WITH UNCONDITIONAL WRAPPER)

***Retrieve an Individual Object***

To retrieve a single JSON value, whether it is simple or complex, you can use the defaults of the `JSON_QUERY` function.

In [ ]:
book = {
  "authors": 
    [
      {"first_name": "Paul",  "last_name" : "Bird"},
      {"first_name": "George","last_name" : "Baklarz"}
    ],
  "foreword": 
    {
      "primary": {"first_name": "Thomas","last_name" : "Hronis"}
    },
  "formats":
    {
      "hardcover": 19.99,
      "paperback":  9.99,
      "ebook"    :  1.99,
      "pdf"      :  1.99
    }
}

In [ ]:
%sql -j VALUES JSON_QUERY(:book,'$.authors');

The results of the function is a JSON array of values as a character string. Retrieving all values from the formats array field results in the following:

In [ ]:
%sql -j VALUES JSON_QUERY(:book,'$.formats' );

If we added wildcard characters to retrieve all of the last_names in the authors object, the `JSON_QUERY` function would return a null value:

In [ ]:
%sql VALUES JSON_QUERY(:book,'$.authors[*].last_name') 

`JSON_QUERY` returns a null value because the path indicates that each possible match is to be returned as an independent result which means that multiple values will be returned in this case and the `ON ERROR` clause is followed (which in this case returns an empty value). You must explicitly state that the results are WRAPPED in an array.

In [ ]:
%sql -j VALUES JSON_QUERY(:book,'$.authors[*].last_name' WITH CONDITIONAL WRAPPER) 

### Quotes
The `JSON_QUERY` function has an option to eliminate the quotes that are required to surround character strings in JSON. 

![JSON_QUERY](images/jq-quotes-clause.png)

There are two options:
* `KEEP QUOTES` – The default is to keep the existing quotes
* `OMIT QUOTES` – Remove a quotation around a string

The `OMIT QUOTES` option is limited to use with the `WITHOUT ARRAY WRAPPER` clause, so multiple values cannot be returned using this keyword. This option is used when you are retrieving a single JSON character value, which by definition must have quotes, and you either want to implicitly convert it to a JSON numeric value (e.g. "123" to 123) or you plan to use the value directly as a character SQL data value.

The previous set of queries against the authors object are shown with the `JSON_QUERY` function modified to `OMIT QUOTES`:

In [ ]:
%sql VALUES JSON_QUERY(:book,'$.authors[0].last_name' WITHOUT ARRAY WRAPPER OMIT QUOTES)

#### Using OMIT QUOTES with JSON_QUERY 

|Path|Description|Result
|:---|:----------|:-----
|`$.authors[*].last_name`|Get all author last names|Null
|`$.authors[0].*`|Get first and last name from author #1|Null
|`$.authors[*].*`|Get all first and last names|Null
|`$.authors[1].last_name`|Last name of author #2|Baklarz
|`$.authors[0]`|Return the entire author object for author #1|`{ "first_name" : "Paul", "last_name" : "Bird" }`

The `OMIT QUOTES` clause does not allow for multiple values to be returned so any JSON path expression with more than one value will result in a null result.

### ON EMPTY and ON ERROR Clause
`JSON_QUERY` has similar `ON EMPTY` and `ON ERROR` clauses as `JSON_VALUE`. The difference between the two functions is that `JSON_QUERY` does not allow for a default value other than an empty object or array. 

![JSON_QUERY](images/jq-empty-clause.png)

Which condition fires is dependent on the use of the `lax` and `strict` keywords and details can be found in the notebook on `JSON_EXISTS`.

The actions for these two error handling clauses are:
* `NULL` – Return a null instead of an error
* `ERROR` – Raise an error
* `EMPTY ARRAY` – Return an empty array
* `EMPTY OBJECT` – Return an empty object

These actions are specified in front of the error handling clause. The default value is `NULL ON EMPTY` and `NULL ON ERROR`. The other option for handling missing values is to return an `EMPTY ARRAY` or an `EMPTY` object. You cannot return a scalar value as a default value.

This SQL will return an empty array when no **middle_name** is found and `strict` is being used for the path expression. 

In [ ]:
%%sql -j
VALUES JSON_QUERY(:book,'strict $.foreword.primary.middle_name' 
   EMPTY ARRAY ON ERROR)

The other alternative is to return an empty object:

In [ ]:
%%sql -j
VALUES JSON_QUERY(:book,'strict $.foreword.primary.middle_name' 
   EMPTY OBJECT ON ERROR) 

If `lax` was used in the expression above, it would raise an `ON EMPTY` condition instead of `ON ERROR`, so the SQL would need to be modified:

In [ ]:
%%sql -j
VALUES JSON_QUERY(:book,'lax $.foreword.primary.middle_name' 
   ERROR ON EMPTY EMPTY OBJECT ON ERROR)

## Summary
The `JSON_VALUE` and `JSON_QUERY` functions provide ways of retrieving individual values or objects from within a document. The `JSON_VALUE` function is used for extracting individual values from a JSON document and returning it as any of the supported Db2 data types, while `JSON_QUERY` is used to retrieve native JSON objects, arrays needed for subsequent JSON data operations.

#### Credits: IBM 2019, George Baklarz [baklarz@ca.ibm.com]